In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)
     
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Wed May 24 11:01:09 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.41.03              Driver Version: 530.41.03    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070         Off| 00000000:01:00.0 Off |                  N/A |
|  0%   48C    P8               17W / 220W|      6MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
need_to_install = False

In [3]:
import sys
sys.executable
sys.path

['/home/jeff/project/poc-project/asl_signs/2nd_place_GISLR',
 '/home/jeff/miniconda3/envs/gislr/lib/python311.zip',
 '/home/jeff/miniconda3/envs/gislr/lib/python3.11',
 '/home/jeff/miniconda3/envs/gislr/lib/python3.11/lib-dynload',
 '',
 '/home/jeff/miniconda3/envs/gislr/lib/python3.11/site-packages']

# Install and Import

In [4]:
if need_to_install:
  !pip install wandb==0.13.3
  !pip install transformers
  !pip install 'git+https://github.com/katsura-jp/pytorch-cosine-annealing-with-warmup'

In [5]:
import sys
code_path = 'code/'

sys.path.insert(0, code_path)

In [6]:
import os
import warnings
warnings.filterwarnings("ignore")

import wandb
import torch
import pandas as pd

import matplotlib.pyplot as plt

from GISLR_utils.utils import get_logger, class2dict
from GISLR_utils.pipeline import train_loop, full_train_loop

# Config

In [7]:
# ====================================================
# Config
# ====================================================
class CFG:
    ####################
    # MAIN
    ####################
    wandb = True
    wandb_project = 'GISLR_IMG_OPTUNA'
    competition = 'G_ISLR_Kaggle'
    wb_group = None
    exp_name = 'exp1'
    base_path = '/home/jeff/project/poc-project/asl_signs/training/'

    seed = 333

    train = True
    LOOP = False
    full_train = True
    debug = False

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    ####################
    # DATA
    ####################
    dataset = 'img_80_mixup' 

    num_workers = 12
    train_bs = 64
    valid_bs = 256 
    n_fold = 8
    trn_fold = [0] 
    fold_group = True
    
    ####################
    # TRAIN
    ####################
    
    early_stopping = 40  # None for dont use
    use_early_break = None # None for dont use
    
    FULL_TRAIN = False 
    apex = True
    
    eval_after = 0
    eval_every = 1
    eval_always_after = 1

    finetune = False
    finetune_path = 'PATH/TO/CKPT'
    finetune_fold = 0
    finetune_sched_opt = True
    finetune_epoch = 2
    finetune_change_seed = True
    
    # Scheduler step 1

    scheduler = 'onecycle'
    onecycle_start = 0.1 
    onecycle_m = 1. 
    num_cycles = 0.5 
    num_warmup_steps = 333

    # Loop step 1

    epochs = 180 

    # LR, optimizer step 1

    eps = 1e-8 
    betas = (0.9, 0.999)
    weight_decay = 0.01
    gradient_accumulation_steps = 1 
    optimizer = "RAdam" 
    
    data_dir = base_path + '/asl_signs/'
    BREAK_EPOCH = 100000
    fill_nan_value = 0.
    
    new_size= (160, 80, 3)
    encoder = 'rexnet_100'
    
    COLAB = False
    drop_bad_inds = False
    deal_with_len = False
    interp_nearest_random = 0.5
    mixup_alpha = 0.5
    k = 5 # topk
    use_restart = False

if True:
    # 訓練的結果會放在results資料夾
    os.makedirs(CFG.base_path + 'results/', exist_ok=True)
    os.makedirs(CFG.base_path + 'results/' + CFG.exp_name, exist_ok=True)
    os.makedirs(CFG.base_path + 'results/' + CFG.exp_name + '/checkpoints', exist_ok=True)
    CFG.save_path = CFG.base_path + 'results/' + CFG.exp_name + '/checkpoints/'
    with open(CFG.base_path + 'results/' + CFG.exp_name + '/CFG.txt', 'w') as f:
        for key, value in CFG.__dict__.items():
            f.write('%s:%s\n' % (key, value))
     

# Load and Prepare Data

In [8]:
import json
def read_dict(file_path):
    path = os.path.expanduser(file_path)
    with open(path, "r") as f:
        dic = json.load(f)
    return dic

train = pd.read_csv(CFG.base_path + 'asl_signs/train.csv')
label_index = read_dict(f"{CFG.base_path}/asl_signs/sign_to_prediction_index_map.json")
index_label = dict([(label_index[key], key) for key in label_index])
train["label"] = train["sign"].map(lambda sign: label_index[sign])
train = train.loc[:9999]
print(train.shape)
display(train.head())

import numpy as np
from sklearn.model_selection import StratifiedGroupKFold, StratifiedKFold

if CFG.fold_group:
    print(f'FOLD SPLIT USING GROUPS')
    split = StratifiedGroupKFold(CFG.n_fold, random_state=42, shuffle=True) #rs = 42

    for k, (_, test_idx) in enumerate(split.split(train, train.sign, groups=train.participant_id)):
        train.loc[test_idx, 'fold'] = k
else:
    print(f'FOLD SPLIT ONLY ON SIGN')
    split = StratifiedKFold(CFG.n_fold, random_state=42, shuffle=True) #rs = 42

    for k, (_, test_idx) in enumerate(split.split(train, train.sign)):
        train.loc[test_idx, 'fold'] = k

train.fold = train.fold.astype(int)
display(train.groupby('fold').size())

display(train['label'].value_counts().to_frame('Class Count').iloc[[0,1,2,3,4, -5,-4,-3,-2,-1]])
print(len(train['label'].unique()))

(10000, 5)


,path,participant_id,sequence_id,sign,label
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow,25
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait,232
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud,48
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird,23
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie,164


FOLD SPLIT USING GROUPS


fold
0    1031
1    1018
2    1395
3     848
4    1371
5    1527
6    1339
7    1471
dtype: int64

,Class Count
31,58
3,56
173,55
0,55
203,55
117,28
112,28
49,27
90,27
21,27


250


# Training

In [9]:
import optuna

def objective(trial):
    """
    Function to optimize model params
    :param trial:(optuna instance) iteration
    :return:(float) metric of model iteration
    """
    
    # CFG.new_size = (128, 120, 3)
    param = {
        'seed': np.random.randint(20, 10000), #6374,
        'aug_prob': trial.suggest_float('aug_prob', 0.15, 0.25), # do_random_affine prob
        'invert_prob': trial.suggest_float('invert_prob', 0.25, 0.32), # it flips all points (hands, lips, pose)
        'scale_prob': trial.suggest_float('scale_prob', 0.17, 0.3), # prob to rescale some parts (e.g. one hand or hand and lips)
        'lr': trial.suggest_float('lr', 2e-3, 2.8e-3),  # LR
        'train_bs': trial.suggest_categorical('train_bs', [32, 64]),  # BS
        'drop_rate': 0.2, #trial.suggest_float('drop_rate', 0.18, 0.22), # model dropout
        'epochs': trial.suggest_int('epochs', 100, 210), # epochs
        'img_masking': 0.98, # prob to use torchaudio masks
        'model': 'img_v0', # model name
        'dataset': 'img_80_mixup', # img_80_onehand for one hand new_size=(T, 64, 3), img_80_mixup for 2 hands new_size=(T, 80, 3)
        'freq_m': trial.suggest_int('freq_m', 66, 80), # mask for time axis  (yes name freq but mask time :) )
        'time_m': trial.suggest_int('time_m', 3, 12), # mask some points. the max possible masked points
        'use_loss_wgt': True, # use wieghted loss
        'pw_bad': 1.45, # power to bad predicted classes
        'pw_com': 0.81, # power to classes which has common classes (e.g. dad, grandpa, grandma)      
        "label_smooth": trial.suggest_float('label_smooth', 0.50, 0.65), # loss label smoothing
        'shift_prob': trial.suggest_float('shift_prob', 0.12, 0.29),  # shift random parts to random value (one or few)
        'mixup_prob': trial.suggest_float('mixup_prob', 0.3, 0.42),  # mixup prob
        'zero_prob': 0., # pixel dropout prob. It didn't work
        'rotate_prob': trial.suggest_float('rotate_prob', 0.18, 0.26),  # rotate one or few parts
        'replace_prob': trial.suggest_float('replace_prob', 0.08, 0.17),  # replace one or two parts from another element with same class
        'deep_supervision': False, # DSV
        'interpol_prob': trial.suggest_float('interpol_prob', 0.15, 0.4), # interploation as in Carno' code
        'normalize': True, # mean std normalization
        'tree_rot_prob': trial.suggest_float('tree_rot_prob', 0.25, 0.55), # finger tree augmentation from Carno's Code
        'interp_nearest_random': trial.suggest_float('interp_nearest_random', 0.35, 0.5),
        'lookahed_k':trial.suggest_int('lookahed_k', 2, 7),
        'lookahed_alpha':trial.suggest_float('lookahed_alpha', 0.3, 0.6),
    }
    print(param)
    CFG.tree_rot_prob = param['tree_rot_prob']
    CFG.interpol_prob = param['interpol_prob']
    CFG.normalize = param['normalize']
    CFG.rotate_prob = param['rotate_prob']    
    CFG.zero_prob = param['zero_prob']
    CFG.replace_prob = param['replace_prob']
    
    CFG.deep_supervision = param['deep_supervision']
    CFG.mixup_prob = param['mixup_prob']
    CFG.shift_prob = param['shift_prob']
    CFG.scale_prob = param["scale_prob"]
    CFG.use_loss_wgt = param['use_loss_wgt'] #False
    CFG.pw_bad = param['pw_bad']
    CFG.pw_com = param['pw_com']
    CFG.img_masking = param["img_masking"]
    CFG.freq_m = param["freq_m"] 
    CFG.time_m = param["time_m"] 
    CFG.scheduler = 'onecycle'  
    CFG.new_size = (160, 80, 3) 
    CFG.loss = 'ce' 
    CFG.alpha = 0.3 
    CFG.model = param['model']
    #CFG.encoder = 'rexnet_100'  # if model == 'timm'
    CFG.aug_prob = param['aug_prob']
    CFG.invert_prob = param['invert_prob']
    CFG.train_bs = param['train_bs']
    CFG.drop_rate = param['drop_rate']
    CFG.lr = param['lr']
    CFG.epochs = param['epochs']
    CFG.num_cycles = 0.5 # param['num_cycles']
    CFG.dataset = param['dataset']
    CFG.seed = param['seed']
    CFG.optimizer = 'Lookahead_RAdam'  # param['optimizer']
    CFG.motion = False
    CFG.use_swa = False
    CFG.swa_start = 0    
    CFG.label_smooth = param['label_smooth'] 
    
    CFG.trn_fold = [0]
    fold_ = CFG.trn_fold[0]
    
    CFG.exp_name = f'EXP_NAME_f{fold_}_{trial.number}_bs{CFG.train_bs}_sm{CFG.label_smooth:.2f}_norm{int(CFG.normalize)}_inter{CFG.interpol_prob:.2f}_time{CFG.time_m}_freq{CFG.freq_m}_repl{CFG.replace_prob:.2f}_lr{CFG.lr:8f}_mixup{CFG.mixup_prob:.2f}_aug{CFG.aug_prob:.2f}_rot{CFG.rotate_prob:.2f}_zero{CFG.zero_prob:.2f}_shift{CFG.shift_prob:.2f}_scale{CFG.scale_prob:.2f}_inver{CFG.invert_prob:.2f}_ep{CFG.epochs}'
    
    os.makedirs(CFG.base_path + 'results/', exist_ok=True)
    os.makedirs(CFG.base_path + 'results/' + CFG.exp_name, exist_ok=True)
    os.makedirs(CFG.base_path + 'results/' + CFG.exp_name + '/checkpoints', exist_ok=True)
    CFG.save_path = CFG.base_path + 'results/' + CFG.exp_name + '/checkpoints/'
    with open(CFG.base_path + 'results/' + CFG.exp_name + '/CFG.txt', 'w') as f:
        for key, value in CFG.__dict__.items():
            f.write('%s:%s\n' % (key, value))

    wandb.init(project='PROJECT_NAME',
            name=CFG.exp_name,
            config=class2dict(CFG),
            group=CFG.wb_group,
            job_type="train",
            dir=CFG.base_path)
    
    LOGGER = get_logger(CFG.base_path + 'results/' + CFG.exp_name + f'/train_f{fold_}')
    acc, topk = train_loop(CFG, train, fold_, LOGGER)
    print(f'FOR PARAMS: {param}')
    print(f'Accuracy: {acc}')
    print(f'TOPK: {topk}')
    print()
    return acc

print('Starting train parameters optimization process.\n'
          f'With main metric Accuracy')
optuna.logging.disable_default_handler()
direct = 'maximize' 
study = optuna.create_study(direction=direct)
study.optimize(objective, n_trials=16)

model_params = study.best_trial.params
print('Best params:')
print(model_params, '\n')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Starting train parameters optimization process.
With main metric Accuracy
{'seed': 2408, 'aug_prob': 0.19721214915170318, 'invert_prob': 0.27540986347265967, 'scale_prob': 0.21340863077529498, 'lr': 0.0025392757819046544, 'train_bs': 64, 'drop_rate': 0.2, 'epochs': 208, 'img_masking': 0.98, 'model': 'img_v0', 'dataset': 'img_80_mixup', 'freq_m': 76, 'time_m': 6, 'use_loss_wgt': True, 'pw_bad': 1.45, 'pw_com': 0.81, 'label_smooth': 0.5649473298767415, 'shift_prob': 0.21126807192102603, 'mixup_prob': 0.3129880475482099, 'zero_prob': 0.0, 'rotate_prob': 0.242177785277867, 'replace_prob': 0.12993976045378228, 'deep_supervision': False, 'interpol_prob': 0.3951466085889599, 'normalize': True, 'tree_rot_prob': 0.4092053694487663, 'interp_nearest_random': 0.470269097367344, 'lookahed_k': 4, 'lookahed_alpha': 0.5190709383090426}


wandb: Currently logged in as: e8035669. Use `wandb login --relogin` to force relogin


========== Fold: 0 training ==========


> SEEDING DONE
Size of train dataset: 8969
Size of valid dataset: 1031
Loaded pretrained weights for efficientnet-b0
Epoch 1/208 | Fold 0


...

ACCURACY SCORE: 0.551891
TOPK SCORE: 0.792435
BASE BETTER
Epoch 70/166 | Fold 0


Ep.70 Train : 100%|█| 280/280 [00:19<00:00, 14.06it/s, _loss=9.54865, global_step=19600, gpu_mem=0.71 GB,
Ep.70 Eval : 100%|████████████████████████| 5/5 [00:01<00:00,  3.67it/s, _loss=10.52666, gpu_mem=1.27 GB]
Epoch 70 - avg_train_loss: 9.54865  avg_val_loss: 10.52666  time: 23s
Epoch 70 - avg_train_loss: 9.54865  avg_val_loss: 10.52666  time: 23s
Epoch 70 - avg_train_loss: 9.54865  avg_val_loss: 10.52666  time: 23s
Epoch 70 - avg_train_loss: 9.54865  avg_val_loss: 10.52666  time: 23s
Epoch 70 - avg_train_loss: 9.54865  avg_val_loss: 10.52666  time: 23s
Epoch 70 - avg_train_loss: 9.54865  avg_val_loss: 10.52666  time: 23s
Epoch 70 - Accuracy: 0.53928 | TopK5: 0.77110
Epoch 70 - Accuracy: 0.53928 | TopK5: 0.77110
Epoch 70 - Accuracy: 0.53928 | TopK5: 0.77110
Epoch 70 - Accuracy: 0.53928 | TopK5: 0.77110
Epoch 70 - Accuracy: 0.53928 | TopK5: 0.77110
Epoch 70 - Accuracy: 0.53928 | TopK5: 0.77110
Score NOT updated. Current best: 0.5567
Score NOT updated. Current best: 0.5567
Score NOT up

ACCURACY SCORE: 0.539282
TOPK SCORE: 0.771096
BASE BETTER
Epoch 71/166 | Fold 0


Ep.71 Train :  53%|▌| 148/280 [00:10<00:09, 13.80it/s, _loss=9.47577, global_step=19748, gpu_mem=0.71 GB,
Trial 5 failed with parameters: {'aug_prob': 0.20274426022885972, 'invert_prob': 0.2570330496566149, 'scale_prob': 0.2021298884184677, 'lr': 0.002579665477681121, 'train_bs': 32, 'epochs': 166, 'freq_m': 73, 'time_m': 12, 'label_smooth': 0.5181705719504381, 'shift_prob': 0.23550656996034908, 'mixup_prob': 0.3642295891925462, 'rotate_prob': 0.2362931794926876, 'replace_prob': 0.10564602086648087, 'interpol_prob': 0.19223805155021637, 'tree_rot_prob': 0.5145124401976287, 'interp_nearest_random': 0.37833743453223007, 'lookahed_k': 7, 'lookahed_alpha': 0.5408197343545651} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/jeff/miniconda3/envs/gislr/lib/python3.11/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_12044/252213303.py

KeyboardInterrupt: 